In [1]:
!pip install sentencepiece
!pip install -U transformers
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.3 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import spacy
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Trainer

# Load SpaCy's NER model for entity extraction
nlp = spacy.load("en_core_web_sm")


def process_clinical_document(document):
    # Extract medical entities using SpaCy's NER model
    doc = nlp(document)
    entities = [(entity.text, entity.label_) for entity in doc.ents]

    # Prompt the user for a query to search within the document
    query = input("Enter a query to search within the document: ")

    # Retrieve relevant sections based on the query
    relevant_sections = []
    for sentence in doc.sents:
        if query in sentence.text.lower():
            relevant_sections.append(sentence.text)

    # Generate a concise and informative summary of the clinical document
    # Load the pretrained BART summarization model
    bart_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base")
    bart_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

    # Prepare input for the summarization model
    inputs = bart_tokenizer(document, return_tensors="pt")

    # Generate the summary
    summary = bart_model.generate(**inputs, max_length=256)
    summary_text = bart_tokenizer.decode(summary[0], skip_special_tokens=True)

    # Print the extracted entities, retrieved sections, and generated summary
    print("Extracted entities:")
    print(entities)

    print("\nRelevant sections:")
    print("\n".join(relevant_sections))

    print("\nSummary:")
    print(summary_text)


In [3]:
# Example usage
clinical_document = """
Patient: Aakash Singh
Age: 23
Diagnosis: Autism Spectrum Disorder (ASD)

Medical History:
- Patient exhibits difficulty with social interaction and communication.
- Patient displays repetitive and restricted behaviors.
- Patient has been receiving behavioral therapy for the past two years.

Current Treatment Plan:
- Continue behavioral therapy sessions.
- Monitor patient's progress and adjust treatment plan as needed.
"""

process_clinical_document(clinical_document)

Enter a query to search within the document: Age


Extracted entities:
[('Aakash Singh\nAge', 'PERSON'), ('23', 'CARDINAL'), ('the past two years', 'DATE')]

Relevant sections:


Summary:
Patient: Aakash Singh, 25Age: 23Gender: MaleDiagnosis: Autism Spectrum Disorder (ASD)Patient Status: Appears to be stable.Medical History:============- Patient exhibits difficulty with social interaction and communication.============–- Patient displays repetitive and restricted behaviors.======- Patient has been receiving behavioral therapy for the past two years.============-Current Treatment Plan:- Continue behavioral therapy sessions.- Monitor patient's progress and adjust treatment plan as needed.============================================================
